In [ ]:
#Lijstje van de benodigde imports
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.dates as mdates

#Omzetten verkregen dataset KNMI naar lijst met gefilterde waardes

# Inladen van de KNMI dataset
input_file = 'KNMI neerslag Wijk aan Zee (2).txt'

#Inladen van de KNMI dataset waarbij de eerste 51 rijen worden verwijderd. Deze informatie is namelijk irrelevant.
#Wel behouden van de kolommen "YYYYMMDD" en "RH". Dit is namelijk relevante informatie 
df_neerslag = pd.read_csv('KNMI neerslag Wijk aan Zee (2).txt', skiprows=8, 
                          skipinitialspace= True, header=0, usecols=["YYYYMMDD","RH"])

#format is voor notatie van de import
df_neerslag['DateTime'] = pd.to_datetime(df_neerslag['YYYYMMDD'].astype(str), format='%Y%m%d')

#Verwijderen NaN rijen
df_neerslag.dropna(inplace=True)

#Omzetten neerslag naar mm, daarna naar m
df_neerslag.RH = df_neerslag.RH * 0.1/1000

# Zet alle waarden kleiner dan 0 op 0
df_neerslag["RH"] = df_neerslag["RH"].apply(lambda x: max(0, x))

#RH waardes in een lijst zetten
Uursom_neerslag = df_neerslag['RH'].tolist()

# Plot de DateTime tegenover de RH
plt.figure(figsize=(12, 6))
plt.plot(df_neerslag["DateTime"], df_neerslag["RH"], label="Neerslag (m)")
plt.title("Neerslag over tijd")
plt.xlabel("Datum")
plt.ylabel("Neerslag (m)")
plt.grid(True)
plt.legend()
plt.show()



#Invoer van KNMI dataset in het model van de waterbergende weg

# Parameters
A_neerslagopp= 100.0  # Oppervlakte van de weg (m^2)
A_bergendopp = 200.0  #Oppervlak van het bergend pakket (m^2)
k_uur = 0.01/24# Doorlatendheidscoëfficiënt van de bodem (m/uur)
por = 0.9
t0 = 0
d_rockwool = 
d_granulaat
por_rockwool

uren = len(Uursom_neerslag)

# Tijdstappen en initiële conditie
dt = 1  # Tijdstap (uren)
t = np.arange(0, uren, dt)  # Tijdstappen van 0 tot 24 uur
V = np.zeros(len(t))  # Array voor de bergingscapaciteit
h = np.zeros(len(t))  # Array voor de waterhoogte in het infiltratiepakket

# Initiële conditie
V[0] = 0.0  # Initiële bergingscapaciteit (m^3)

# Numerieke oplossing met behulp van de methode van Euler
for i in range(1, len(t)):
    dVdt = (Uursom_neerslag[i] * A_neerslagopp - k_uur * A_bergendopp)
    V[i] = max(0, V[i - 1] + dVdt * dt)
    h[i] = max(0, V[i] / por / A_bergendopp)

# Plot de numerieke oplossing voor het volume (verbeterde x-as)
plt.figure(figsize=(20, 10))
plt.plot(df_neerslag["DateTime"], V, label='Numerieke Oplossing (m^3)', marker='o', markersize=4, linestyle='-')
plt.xlabel('Tijd (jaren)')  # Verander x-as label naar 'Tijd (jaren)'
plt.ylabel('Bergingscapaciteit (m^3)')
plt.title('Numerieke Oplossingen van de Bergingscapaciteit met Dagelijkse Neerslagsom')
plt.legend()
plt.grid(True)
plt.gca().xaxis.set_major_locator(mdates.YearLocator(base=1))  # Alleen de jaren laten zien
plt.gca().xaxis.set_major_formatter(mdates.DateFormatter('%Y'))
plt.xticks(rotation=45)  # Roteren van de waardes op de x-as
plt.show()

# Plot de numerieke oplossing voor waterhoogte
plt.figure(figsize=(20, 10))
plt.plot(t, h, label='Numerieke Oplossing (m)', marker='o', markersize=4, linestyle='-')
plt.xlabel('Tijd (uur)')
plt.ylabel('Waterhoogte (m)')
plt.title('Numerieke Oplossing van de Waterhoogte met Piekbelasting')
plt.legend()
plt.grid(True)
plt.xticks(np.arange(0, 2, 1))  # Tijdstapmarkeringen om de 1 dag
plt.show()
